In [1]:
import os
import cv2
import time
import numpy as np
import face_recognition
from scipy import spatial
from skimage import feature
import matplotlib.pyplot as plt

In [2]:
aug = True
if aug:
    path = 'datasets/augmented'
    file_encodings = 'encodings_aug.npy'
    predict_save_name = 'predicted_compare_aug.jpg'
    file_names = 'names.npy'
    dict_cosine = 'dict_cosine.npy'
else:
    path = 'datasets/single'
    file_encodings = 'encodings.npy'
    predict_save_name = 'predicted_compare.jpg'
    file_names = 'names.npy'

threshold = 0.95

In [ ]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(
            image, self.numPoints, self.radius, method="uniform").astype("uint8")

        hist = cv2.calcHist([lbp], [0], None, [
                            self.numPoints+2], [0, self.numPoints+2]).flatten()
        #hist = np.histogram(lbp.ravel(), bins=range(0,self.numPoints+3), range=(0,self.numPoints+2))[0]

        hist = hist.astype("float")
        hist /= (hist.sum()+eps)

        return hist

In [ ]:
desc = LocalBinaryPatterns(24, 3)

In [27]:
def TIME(f):
    def wrapper(*args):
        start_time = time.time()
        res = f(*args)
        end_time = time.time()
        print('Run-time: %fs\n' % (end_time-start_time))
        return res
    return wrapper


@TIME
def findEncodings():
    encodings, names = [], []
    for person in os.listdir(path):
        if person != '.DS_Store':
            image_dir = path + '/' + person
            pix = os.listdir(image_dir)
            if not os.path.isdir(image_dir):
                os.mkdir(image_dir)
            # Loop through each training image for the current person
            for person_img in pix:
                # Get the face encodings for the face in each image file
                face = face_recognition.load_image_file(
                    image_dir + "/" + person_img)

                face_enc = face_recognition.face_encodings(face)[0]
                # Add face encoding for current image with corresponding label (name) to the training data
                encodings.append(face_enc)
                names.append(person)
    print('\nNumber of classes: %d\n' % len(names))
    with open(file_encodings, 'wb') as f:
        np.save(f, encodings)
    with open(file_names, 'wb') as f:
        np.save(f, names)
    print('Encodings Complete')
    return np.array(encodings), np.array(names)


@TIME
def importFaceEncoded(filename):
    # with open(filename, 'rb') as f:
    encodeListKnown = np.load(filename)
    print(filename + ' has imported!')
    return encodeListKnown


def face_distance(face_encodings, face_to_compare):
    if len(face_encodings) == 0:
        return np.empty((0))
    # normed = np.linalg.norm(face_encodings - face_to_compare, axis=1)
    # return normed.T * normed
    #----
    arr = []
    
    for face_encoding in face_encodings:
        sum=0
        for i, j in zip (face_encoding, face_to_compare):
            sum += (i-j)*(i-j)
        # print(np.sqrt(sum))
        # arr.append(np.sqrt(sum))
        arr.append(spatial.distance.cosine(face_encoding, face_to_compare))

    return np.array(arr)

def draw_box(img, face_encodings, face_locations):
    fontFace = cv2.FONT_HERSHEY_COMPLEX
    count, fontScale, thickness = 0, 0.5, 1
    red, green = (255, 0, 0), (0, 255, 0)
    for encodeFace, faceLoc in zip(face_encodings, face_locations):
        tolerance = 0.6
        faceDis = face_distance(encodings, encodeFace)
        matchIndex = np.argmin(faceDis)
        confidence = 1 - faceDis[matchIndex]
        if faceDis[matchIndex] <= 1:
            color = green
            if confidence >= 0:
                name = '%s %.4f' % (
                    names[matchIndex], confidence)
                count += 1
            else:
                name = 'Unknown~'
                color = red
            y1, x2, y2, x1 = faceLoc
            labelSize = cv2.getTextSize(
                name, fontFace, fontScale, 2)[0]
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
            cv2.rectangle(
                img, (x1, y1), (x1+labelSize[0], y1-labelSize[1]-20), color, cv2.FILLED)
            cv2.putText(img, name, (x1, y1-10),
                        fontFace, fontScale, (0, 0, 0), thickness)
    cnt_x1, cnt_y1 = 12, 12
    count_students = '%d student(s)' % count
    countSize = cv2.getTextSize(
        count_students, fontFace, fontScale, thickness)[0]
    cv2.rectangle(img, (cnt_x1, cnt_y1), (cnt_y1+countSize[0]+18, cnt_y1+countSize[1]+18),
                  green, -1)
    cv2.putText(img, count_students, (cnt_x1+8, countSize[1]+cnt_y1+8), fontFace,
                fontScale=fontScale, color=(0, 0, 0), thickness=thickness)
    return img

@TIME
def predict():
    filename = 'validation/thaovatinh.jpg'
    img = face_recognition.load_image_file(filename)
    face_locations = face_recognition.face_locations(img)
    face_encodings = face_recognition.face_encodings(img, face_locations)
    img = draw_box(img, face_encodings, face_locations)
    plt.imsave(predict_save_name, img)
    print('Predcited')

In [28]:
replace = False
if not os.path.isfile(file_encodings) or replace:
    encodings, names = findEncodings()
else:
    encodings = importFaceEncoded(file_encodings)
    names = importFaceEncoded(file_names)
    # mapping = importFaceEncoded(dict_cosine)

encodings_aug.npy has imported!
Run-time: 0.001597s

names.npy has imported!
Run-time: 0.001961s



In [29]:
# print(names)

In [30]:
predict()

Predcited
Run-time: 1.987148s

